### Plot candles with a list of moving averages

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import utils
import plotly.graph_objects as go

In [2]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [8,16,32,64,128,256]

In [3]:
df = pd.read_pickle(utils.get_data_filename(pair, granularity, True))

In [4]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [5]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [6]:
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [7]:
df_ma.dropna(inplace=True)

In [8]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_8,MA_16,MA_32,MA_64,MA_128,MA_256
255,2024-01-11T19:00:00.000000000Z,1.09651,1.09830,1.09650,1.09820,1.095804,1.096701,1.096659,1.095190,1.094864,1.098440
256,2024-01-11T20:00:00.000000000Z,1.09820,1.09826,1.09562,1.09715,1.095650,1.096661,1.096746,1.095214,1.094879,1.098413
257,2024-01-11T21:00:00.000000000Z,1.09714,1.09754,1.09702,1.09736,1.096075,1.096631,1.096849,1.095251,1.094900,1.098386
258,2024-01-11T22:00:00.000000000Z,1.09694,1.09762,1.09694,1.09730,1.096310,1.096551,1.096952,1.095287,1.094921,1.098358
259,2024-01-11T23:00:00.000000000Z,1.09729,1.09828,1.09708,1.09824,1.096760,1.096572,1.096999,1.095327,1.094942,1.098335


In [9]:
df_plot = df_ma.iloc[-300:].copy()

In [10]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f'MA_{ma}'
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
    ))
fig.update_layout(width=1200,height=600,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()